The first chunk of code is borrrowed from GeeksforGeeks.
https://www.geeksforgeeks.org/web-scraping-financial-news-using-python/

It helped me remember basics of HTML, like the the < a > tag is used to define a hyperlink, href holds the url to the hyperlink etc.

By parsing through all < a > tags, picked the bs4 element that was holding the title of news articles and printed them out.

Though I ended up not using it later, kept here as trail of how self thought process evolved 

---



In [1]:
# Import Required Libraries
from bs4 import BeautifulSoup as BS
import requests as req
 
url = "https://www.coindesk.com/business/"

# 
webpage = req.get(url)
traverse = BS(webpage.content, "html.parser")
M = 1
for link in traverse.find_all('a'):
   

    if(str(type(link.string)) == "<class 'bs4.element.NavigableString'>"
       and len(link.string) > 35):
 
        print(str(M)+".", link.string)
        M += 1

1. Basic Attention Token Price Index (BTX)
2. Super-Early Bird Extended Until Thurs!
3. Crypto Trading Platform Bitget to Offer Off-Exchange Settlement With Copper’s ClearLoop
4. Bitpanda Partners With Coinbase to Help Europe’s Banks Offer Crypto to Customers
5. Bitcoin Payments Firm Strike’s Headquarters to Stay in U.S., Despite New El Salvador Office
6. AI Can Generate a Trading Edge in Crypto Markets
7. Bitfinex Invests in Chilean Crypto Exchange OrionX to Expand Presence in Latin America
8. Voltz Protocol Brings a Wall Street Rates Stalwart to DeFi
9. Layer 1 Blockchain Tenet Partners With Conflux and Qtum for More Exposure in China
10. TradFi Hedge Fund Hunting Hill Starts Crypto Arm
11. AI Can Generate a Trading Edge in Crypto Markets
12. Crypto’s Quirky Automated Market Makers and How They Differ From TradFi Exchanges
13. Ethereum Staking Needs Its (Honest) LIBOR
14. Full Transcript: NY Times' Interview With Sam Bankman-Fried


Now to understand Transformers would take much longer here .
But in brief they are pretrained semi-supervised ML models used primarily in NLP applications

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [12]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Setup Summarization Model**

PegasusTokenizer will simply split up sentences into words or tokens. Though tokens can be sub words or individual characters as well.

I've also used this in the past hence best choice for here too

In [4]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'human-centered-summarization/financial-summarization-pegasus'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

**Summarize a Single Article**

In [6]:
url = "https://finance.yahoo.com/news/two-fed-officials-more-interest-143200569.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [7]:
paragraphs[3]

<p><a class="link" data-ylk="slk:Mexico Raises Alert Level on Volcano Rumbling Near Capital;elm:context_link;itc:0" href="https://www.bloomberg.com/news/articles/2023-05-21/mexico-raises-alert-level-on-popo-volcano-near-mexico-city?utm_campaign=bn&amp;utm_medium=distro&amp;utm_source=yahooUS" rel="nofollow noopener" target="_blank">Mexico Raises Alert Level on Volcano Rumbling Near Capital</a></p>

In [13]:
paragraphs[3].get_text()

'Mexico Raises Alert Level on Volcano Rumbling Near Capital'

In [11]:
paragraphs[3].string


'Mexico Raises Alert Level on Volcano Rumbling Near Capital'

In [8]:
paragraphs[3].text

'Mexico Raises Alert Level on Volcano Rumbling Near Capital'

The linked webpage summarises soup's syntax use in very detailed way
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [14]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [15]:
ARTICLE

'(Bloomberg) -- Two Federal Reserve hawks saw the need to raise interest rates further this year, days after Chair Jerome Powell signaled a pause in June, while two other officials voiced support for patience. Most Read from Bloomberg China’s $23 Trillion Local Debt Mess Is About to Get Worse Mexico Raises Alert Level on Volcano Rumbling Near Capital Parents Sue Elite Schools for ‘Indoctrinating’ Their Kids With Anti-Racist Policies China’s New Covid Wave Set to See 65 Million Cases a Week Federal Reserve Bank of St. Louis President James Bullard said Monday that he backed two more increases and his Minneapolis colleague Neel Kashkari said if the central bank does pause, it should signal tightening isn’t over. Their comments — while not explicitly pushing back against holding fire at the Federal Open Market Committee’s June 13-14 meeting — bolster the case aired by other officials that they might skip a move next month, while keeping the door open to hiking further if needed to quell i

In [16]:
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt')
output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [17]:
summary

'St. Louis’ Bullard sees two more increases this year. Minneapolis’ Kashkari sees no need to raise rates in June'

**Building a News and Sentiment Pipeline**

In [36]:
Terms_of_Interest = ['TSLA', 'InterestRates', 'DebtCeiling']

**Search for Stock News using Google and Yahoo Finance**

In [38]:
# def search_for_stock_news_urls(ticker):
#     search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)


#     r = requests.get(search_url)
#     soup = BeautifulSoup(r.text, 'html.parser')
#     atags = soup.find_all('a')
#     hrefs = [link['href'] for link in atags]
#     return hrefs 

In [39]:
def find_relevant_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws&tbs=qdr:d".format(ticker)


    r = requests.get(search_url)

    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs 

In the above chunk , the above website guided me to which google serach tags to be used to get just News and specify last day as date range
https://stenevang.wordpress.com/2013/02/22/google-advanced-power-search-url-request-parameters/

In [40]:
raw_urls = {ticker:find_relevant_urls(ticker) for ticker in Terms_of_Interest}

**Strip out unwanted URLs**

In [41]:
import re

In [42]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [43]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [44]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in Terms_of_Interest}

In [28]:
#Search and Scrape Cleaned URLs

In [45]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [46]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in Terms_of_Interest}

Thanks to https://github.com/nicknochnack/Stock-and-Crypto-News-ScrapingSummarizationSentiment
for code inspiration and guidance

Sumarize all articles

In [47]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [48]:
summaries = {ticker:summarize(articles[ticker]) for ticker in Terms_of_Interest}
summaries

{'TSLA': ['All images are copyrighted.',
  'XPeng sees Q2 deliveries falling 36.1% to 39.0% from a year ago. China EV market tight, uneven recovery following CO-19 protocols',
  "The centibillionaire CEO of Tesla, Twitter, and SpaceX spoke to WSJ. Musk says his children won't inherit stakes in his companies",
  'We are aware of the issue and are working to resolve it.',
  'Florida governor is a ‘meddlesome,’ ‘woke’ governor. ‘DeSantis fatigue’ may be giving voters pause on his policies',
  'British EV startup to focus on U.S. market. 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884',
  'We are aware of the issue and are working to resolve it.',
  'Musk says the token is his preferred cryptocurrency. Dogecoin currently trades at just over 7 cents, 90% below all-time high',
  '‘There should be

I don't like certain text like 'All images are copyrighted' etc

In [63]:
dict1 = {}
for key, value in summaries.items():
    value = list(value)
    if 'All images are copyrighted.' in value:
        value.remove('All images are copyrighted.')
        summaries[key] = value
dict1 = summaries
dict1

{'TSLA': ['XPeng sees Q2 deliveries falling 36.1% to 39.0% from a year ago. China EV market tight, uneven recovery following CO-19 protocols',
  "The centibillionaire CEO of Tesla, Twitter, and SpaceX spoke to WSJ. Musk says his children won't inherit stakes in his companies",
  'We are aware of the issue and are working to resolve it.',
  'Florida governor is a ‘meddlesome,’ ‘woke’ governor. ‘DeSantis fatigue’ may be giving voters pause on his policies',
  'British EV startup to focus on U.S. market. 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884',
  'We are aware of the issue and are working to resolve it.',
  'Musk says the token is his preferred cryptocurrency. Dogecoin currently trades at just over 7 cents, 90% below all-time high',
  '‘There should be a significant third horse in the

In [65]:
dict2 = {}
for key, value in dict1.items():
    value = list(value)
    if 'We are aware of the issue and are working to resolve it.' in value:
        value.remove('We are aware of the issue and are working to resolve it.')
        dict1[key] = value
dict2 = dict1
dict2

{'TSLA': ['XPeng sees Q2 deliveries falling 36.1% to 39.0% from a year ago. China EV market tight, uneven recovery following CO-19 protocols',
  "The centibillionaire CEO of Tesla, Twitter, and SpaceX spoke to WSJ. Musk says his children won't inherit stakes in his companies",
  'Florida governor is a ‘meddlesome,’ ‘woke’ governor. ‘DeSantis fatigue’ may be giving voters pause on his policies',
  'British EV startup to focus on U.S. market. 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884 888-349-8884',
  'Musk says the token is his preferred cryptocurrency. Dogecoin currently trades at just over 7 cents, 90% below all-time high',
  '‘There should be a significant third horse in the race,’ he says. Tesla would definitely win if it built self-driving cars, he says',
  'We are aware of the issue and are wo

Now that the data is cleaned, lets add onto it further

In [67]:
!pip install gTTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
mytext = str(dict2)

In [69]:
from gtts import gTTS
  
import os

# mytext = 'Long Pause Vijayant'
  
# Language in which you want to convert
language = 'en'

myobj = gTTS(text=mytext, lang=language, slow=True)
  


This page guided me that I can also get an Irish English accent here. 
Why are we doing this? Because we can!

In [70]:
myobj = gTTS(text=mytext, lang=language, tld = 'ie', slow=True)

In [71]:
# Saving the converted audio in a mp3 file named
# welcome 
myobj.save("welcome.mp3")

In [72]:
import IPython
IPython.display.Audio("welcome.mp3")

Wondering how does the Indian English accent sounds :P

In [73]:
myobj2 = gTTS(text=mytext, lang=language, tld = 'co.in', slow=True)
myobj2.save("welcome.mp3")
IPython.display.Audio("welcome.mp3")

Beautiful! An evening spent well